In [1]:
# import libraries

import requests
import time
import datetime
import pandas as pd

In [2]:
# set up an empty list to combine gathered information

posts = []

In [6]:
# create a function to pull the information from reddit

url_submission = "https://api.pushshift.io/reddit/search/submission"
url_comments = "https://api.pushshift.io/reddit/search/comment"

def reddit(url, params):

    # set how many times we would pull from reddit - the limit is 100 now
    while params['after'] < params['before']:

        # get the webpage status
        response = requests.get(url, params = params)

        # check the status and pull if it is 200
        if response.status_code == 200:
            
            # get the info from JSON and name value of data as post 
            data = response.json()
            post = data['data']
            
            # extend the data to posts
            posts.extend(post)
            
            # get the data "after" the UTC of last post of last pull
            params['after'] = post[-1]['created_utc']    
            
            # check if we reached the date 5/15/2020
            if params['after'] >= 1571184000:
                print('We have reached the last pull')
                break
            else:
                # make sure it is still running
                print("Still running.")
                continue
            
            # extend the data to posts
            posts.extend(post)
            
        else:
            print(f'An error: {response.status_code}')
            break
               
        # set 2 seconds to prevent from locking out
        time.sleep(2)

In [7]:
params = {
          'subreddit': ['bipolarreddit', 'depression', 'Anxiety', 'healthanxiety', 'SuicideWatch', 
                        'lonely', 'alcoholism', 'mentalhealth'],
          'size': 100,
          # we are interested in any post from 9/16/2019 to 6/15/2020
          'after': 1568613599,  # 9/15/2019 23:59:59
          'before': 1571184000  # 10/16/2020 00:00:00 
                                # --> will have to do this in steps, otherwise will get error code 429
         }

In [8]:
# call in the function - submission

reddit(url_submission, params)

An error: 429


In [ ]:
# call in the function - comments

reddit(url_comments, params, 3)

In [ ]:
# check the length of posts

len(posts)

In [ ]:
# make it into a dataframe

df = pd.DataFrame(posts)

In [ ]:
# save it into a csv

df.to_csv('../data/original.csv', index = False)